In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb

import numpy as np
import random
from tqdm import tqdm
from glob import glob

## 1. 데이터 불러오기

In [2]:
path = './data/'

train = pd.read_csv(path + 'train_features.csv')
train_label = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

## 3. 데이터 전처리

In [3]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
ml_train = train[features].groupby('id').agg(['max', 'min', 'mean'])
ml_test = test[features].groupby('id').agg(['max', 'min', 'mean'])

In [4]:
ml_train.shape, ml_test.shape

((3125, 18), (782, 18))

In [5]:
ml_label = train_label['label']

## 4. ML 모델링

In [6]:
import gc

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [7]:
# 모델 1번: xgboost

def build_xgboost(split_num, train, target, test, rnd):
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 12,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                }
    
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train.iloc[train_idx]
        y = target.iloc[train_idx]
        valid_x = train.iloc[val_idx]
        valid_y = target.iloc[val_idx]

        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_temp = xgb.DMatrix(valid_x)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        train_pred[val_idx] = model.predict(d_temp)
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

xgb_train1, xgb_test1 = build_xgboost(5, ml_train, ml_label, ml_test, 1)
xgb_train2, xgb_test2 = build_xgboost(5, ml_train, ml_label, ml_test, 2)

[0]	train-mlogloss:3.62825	valid-mlogloss:3.67101
[100]	train-mlogloss:0.28305	valid-mlogloss:1.21516
[200]	train-mlogloss:0.08127	valid-mlogloss:1.10142
[300]	train-mlogloss:0.04497	valid-mlogloss:1.09137
[361]	train-mlogloss:0.03647	valid-mlogloss:1.08973
------------------
[0]	train-mlogloss:3.61803	valid-mlogloss:3.65704
[100]	train-mlogloss:0.27679	valid-mlogloss:1.28499
[200]	train-mlogloss:0.07877	valid-mlogloss:1.19083
[300]	train-mlogloss:0.04396	valid-mlogloss:1.18798
[319]	train-mlogloss:0.04090	valid-mlogloss:1.18980
------------------
[0]	train-mlogloss:3.62390	valid-mlogloss:3.64505
[100]	train-mlogloss:0.28280	valid-mlogloss:1.18239
[200]	train-mlogloss:0.08126	valid-mlogloss:1.08809
[299]	train-mlogloss:0.04555	valid-mlogloss:1.08363
------------------
[0]	train-mlogloss:3.62030	valid-mlogloss:3.65042
[100]	train-mlogloss:0.28112	valid-mlogloss:1.21359
[200]	train-mlogloss:0.08108	valid-mlogloss:1.11948
[300]	train-mlogloss:0.04497	valid-mlogloss:1.11898
[326]	train-mlo

In [8]:
# 모델 2번: catboost

def build_catboost(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train.iloc[train_idx]
        y = target.iloc[train_idx]
        valid_x = train.iloc[val_idx]
        valid_y = target.iloc[val_idx]

        model = cb.CatBoostClassifier(iterations=1500,
                                      learning_rate=0.01,
                                      l2_leaf_reg=3.5,
                                      depth=8,
                                      rsm=0.98,
                                      loss_function= 'MultiClass',
                                      eval_metric='AUC',
                                      use_best_model=True,
                                      random_seed=42,
                                      verbose=50)

        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds=30)
        
        # save feat
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

catboost_train1, catboost_test1 = build_catboost(5, ml_train, ml_label, ml_test, 1)
catboost_train2, catboost_test2 = build_catboost(5, ml_train, ml_label, ml_test, 2)

0:	test: 0.8788202	best: 0.8788202 (0)	total: 546ms	remaining: 13m 38s
50:	test: 0.9751737	best: 0.9751737 (50)	total: 22.4s	remaining: 10m 37s
100:	test: 0.9781930	best: 0.9781930 (100)	total: 44.2s	remaining: 10m 11s
150:	test: 0.9802737	best: 0.9802737 (150)	total: 1m 5s	remaining: 9m 48s
200:	test: 0.9810318	best: 0.9811934 (196)	total: 1m 27s	remaining: 9m 26s
250:	test: 0.9817006	best: 0.9818524 (238)	total: 1m 49s	remaining: 9m 4s
300:	test: 0.9824387	best: 0.9824387 (300)	total: 2m 11s	remaining: 8m 42s
350:	test: 0.9826683	best: 0.9827424 (348)	total: 2m 33s	remaining: 8m 20s
400:	test: 0.9832590	best: 0.9834020 (398)	total: 2m 54s	remaining: 7m 58s
450:	test: 0.9836716	best: 0.9837902 (431)	total: 3m 16s	remaining: 7m 37s
500:	test: 0.9843569	best: 0.9843569 (500)	total: 3m 38s	remaining: 7m 15s
550:	test: 0.9846588	best: 0.9848202 (539)	total: 3m 59s	remaining: 6m 52s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9848201669
bestIteration = 539

Shrink m

100:	test: 0.9783585	best: 0.9784273 (95)	total: 42.7s	remaining: 9m 51s
150:	test: 0.9792417	best: 0.9794221 (133)	total: 1m 3s	remaining: 9m 29s
200:	test: 0.9804051	best: 0.9804151 (197)	total: 1m 25s	remaining: 9m 9s
250:	test: 0.9814011	best: 0.9814011 (250)	total: 1m 46s	remaining: 8m 48s
300:	test: 0.9818473	best: 0.9821082 (295)	total: 2m 7s	remaining: 8m 27s
350:	test: 0.9824234	best: 0.9824234 (350)	total: 2m 28s	remaining: 8m 6s
400:	test: 0.9830334	best: 0.9830731 (393)	total: 2m 49s	remaining: 7m 44s
450:	test: 0.9833647	best: 0.9834558 (425)	total: 3m 10s	remaining: 7m 23s
500:	test: 0.9839403	best: 0.9840245 (496)	total: 3m 32s	remaining: 7m 3s
550:	test: 0.9842076	best: 0.9842622 (549)	total: 3m 53s	remaining: 6m 42s
600:	test: 0.9847087	best: 0.9847662 (594)	total: 4m 14s	remaining: 6m 21s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9848129314
bestIteration = 608

Shrink model to first 609 iterations.
------------------
0:	test: 0.8798594	best: 

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [13]:
# # https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# # 데이터 증강을 통해 과접합을 줄여보자 - DACON.

# x_train = []

# for uid in tqdm(train['id'].unique()):
#     temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
#     x_train.append(temp)

# x_train = np.array(x_train, np.float32)
# x_train = x_train[:,:,:,np.newaxis]

# y_train = train_label['label']

# x_test = []

# for uid in tqdm(test['id'].unique()):
#     temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
#     x_test.append(temp)

# x_test = np.array(x_test, np.float32)
# x_test = x_test[:,:,:,np.newaxis]

In [14]:
def aug(data, shift):
    shift_data = np.roll(data, shift, axis=2)
    return shift_data

# # 데이터 증강
# shift_data = []
# shift_label = []
# for n in tqdm(range(100)):
#     shifted = aug(x_train, n*6)
#     shift_data.append(shifted)
#     shift_label.append(y_train)

In [15]:
x_train = np.array(train.iloc[:,2:]).reshape(-1, 600, 6, 1)
y_train = tf.keras.utils.to_categorical(train_label['label'])
x_test = np.array(test.iloc[:,2:]).reshape(-1, 600, 6, 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(3125, 600, 6, 1)
(3125, 61)
(782, 600, 6, 1)


In [16]:
# 모델 3번: lstm

# def build_lstm(split_num, train, target, test, rnd):
#     # return train pred prob and test pred prob 
#     train_pred, test_pred = np.zeros((x_train.shape[0], 61)), np.zeros((x_test.shape[0], 61))
    
#     es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
#     mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for train_idx, val_idx in mskf.split(train, target):

#         # split train, validation set
#         X = train[train_idx]
#         y = target[train_idx]
#         valid_x = train[val_idx]
#         valid_y = target[val_idx]

#         #가벼운 모델 생성
#         model = Sequential()
#         model.add(LSTM(32, input_shape=(600,6)))
#         model.add(Dense(128, activation='relu'))
#         model.add(Dense(61, activation='softmax'))

#         model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#         model.fit(X,y,
#                   epochs=100,
#                   batch_size=64,
#                   validation_data=[valid_x,valid_y],
#                   callbacks=[es]
#                  )
        
#         # save feat
#         train_pred[val_idx] = model.predict(valid_x)
#         test_pred += model.predict(test)/split_num
        
#         # release
#         del model
#         gc.collect()
#         print('------------------')
        
#     return train_pred, test_pred

# lstm_train1, lstm_test1 = build_lstm(5, x_train, y_train, x_test, 1)
# lstm_train2, lstm_test2 = build_lstm(5, x_train, y_train, x_test, 2)

In [17]:
# 모델 4번: cnn

def build_cnn(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((x_train.shape[0], 61)), np.zeros((x_test.shape[0], 61))

    es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in mskf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        #가벼운 모델 생성
        model = Sequential()

        model.add(Conv2D(input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]), filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
        model.add(Activation('relu'))
        model.add(Conv2D(filters = 512, kernel_size = (3,3), strides = (1,1), padding = 'same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size = (2,2)))
        model.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
        model.add(Activation('relu'))
        model.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size = (2,2)))

        # prior layer should be flattend to be connected to dense layers
        model.add(Flatten())
        # dense layer with 50 neurons
        model.add(Dense(128, activation = 'relu'))
        # final layer with 3 neurons to classify the instances
        model.add(Dense(61, activation = 'softmax'))

        adam = optimizers.Adam(lr = 0.001)
        model.compile(optimizer = adam, metrics = ['accuracy'], loss = 'categorical_crossentropy')
        model.fit(X,y, epochs=100, batch_size=128, validation_split=0.2)
        
        # save feat
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

cnn_train1, cnn_test1 = build_cnn(5, x_train, y_train, x_test, 1)
cnn_train2, cnn_test2 = build_cnn(5, x_train, y_train, x_test, 2)

c:\users\pc\.conda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train on 2000 samples, validate on 500 samples
Epoch 1/100
2000/2000 [==============================] - 727s 363ms/sample - loss: 103.7604 - accuracy: 0.2610 - val_loss: 116.9293 - val_accuracy: 0.3580
Epoch 2/100
2000/2000 [==============================] - 2s 996us/sample - loss: 101.4695 - accuracy: 0.2395 - val_loss: 157.4118 - val_accuracy: 0.2200
Epoch 3/100
2000/2000 [==============================] - 2s 993us/sample - loss: 11.1235 - accuracy: 0.4680 - val_loss: 181.7101 - val_accuracy: 0.2820
Epoch 4/100
2000/2000 [==============================] - 2s 988us/sample - loss: 8.8765 - accuracy: 0.4805 - val_loss: 162.3142 - val_accuracy: 0.1360
Epoch 5/100
2000/2000 [==============================] - 2s 994us/sample - loss: 7.8309 - accuracy: 0.4660 - val_loss: 130.8460 - val_accuracy: 0.2840
Epoch 6/100
2000/2000 [==============================] - 2s 994us/sample - loss: 11.2900 - accuracy: 0.4650 - val_loss: 133.9566 - val_accuracy: 0.3160
Epoch 7/100
2000/2000 [================

2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7873 - accuracy: 0.4850 - val_loss: 7.0376 - val_accuracy: 0.3260
Epoch 56/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7867 - accuracy: 0.4850 - val_loss: 7.0802 - val_accuracy: 0.3260
Epoch 57/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7882 - accuracy: 0.4850 - val_loss: 7.1225 - val_accuracy: 0.3260
Epoch 58/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7753 - accuracy: 0.4850 - val_loss: 7.1650 - val_accuracy: 0.3260
Epoch 59/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7935 - accuracy: 0.4850 - val_loss: 7.2087 - val_accuracy: 0.3260
Epoch 60/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7848 - accuracy: 0.4850 - val_loss: 7.2517 - val_accuracy: 0.3260
Epoch 61/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7821 - accuracy: 0.4850 - val_loss: 7.2954 

Epoch 10/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 26.2811 - accuracy: 0.4635 - val_loss: 3.8805 - val_accuracy: 0.3880
Epoch 11/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 16.6992 - accuracy: 0.4715 - val_loss: 3.9980 - val_accuracy: 0.3560
Epoch 12/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 17.0035 - accuracy: 0.4825 - val_loss: 5.3305 - val_accuracy: 0.1900
Epoch 13/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 15.5188 - accuracy: 0.4630 - val_loss: 7.7052 - val_accuracy: 0.0780
Epoch 14/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 8.6869 - accuracy: 0.4695 - val_loss: 9.7718 - val_accuracy: 0.0500
Epoch 15/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 6.3173 - accuracy: 0.4675 - val_loss: 13.1489 - val_accuracy: 0.0480
Epoch 16/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.8133 - accuracy: 0.4765 -

Epoch 65/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8002 - accuracy: 0.4825 - val_loss: 14.7002 - val_accuracy: 0.3180
Epoch 66/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8101 - accuracy: 0.4825 - val_loss: 14.8876 - val_accuracy: 0.3180
Epoch 67/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8079 - accuracy: 0.4825 - val_loss: 15.0741 - val_accuracy: 0.3180
Epoch 68/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8644 - accuracy: 0.4795 - val_loss: 15.2977 - val_accuracy: 0.3180
Epoch 69/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8068 - accuracy: 0.4825 - val_loss: 15.4568 - val_accuracy: 0.3180
Epoch 70/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8107 - accuracy: 0.4825 - val_loss: 15.6420 - val_accuracy: 0.3180
Epoch 71/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8009 - accuracy: 0.4825 

...

Epoch 73/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0502 - accuracy: 0.4815 - val_loss: 22.8188 - val_accuracy: 0.3540
Epoch 74/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.1243 - accuracy: 0.4815 - val_loss: 22.5845 - val_accuracy: 0.3540
Epoch 75/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.1193 - accuracy: 0.4645 - val_loss: 22.3132 - val_accuracy: 0.1820
Epoch 76/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.1178 - accuracy: 0.4630 - val_loss: 22.0254 - val_accuracy: 0.1820
Epoch 77/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0910 - accuracy: 0.4630 - val_loss: 21.7494 - val_accuracy: 0.1820
Epoch 78/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0871 - accuracy: 0.4630 - val_loss: 21.4573 - val_accuracy: 0.1820
Epoch 79/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0469 - accuracy: 0.4635 

2000/2000 [==============================] - 2s 1ms/sample - loss: 3.1158 - accuracy: 0.4805 - val_loss: 3.8117 - val_accuracy: 0.5060
Epoch 28/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0820 - accuracy: 0.4805 - val_loss: 3.8029 - val_accuracy: 0.5060
Epoch 29/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0450 - accuracy: 0.4805 - val_loss: 3.7944 - val_accuracy: 0.5060
Epoch 30/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 3.0140 - accuracy: 0.4805 - val_loss: 3.7864 - val_accuracy: 0.5060
Epoch 31/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.9962 - accuracy: 0.4805 - val_loss: 3.7788 - val_accuracy: 0.5060
Epoch 32/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.9636 - accuracy: 0.4805 - val_loss: 3.7716 - val_accuracy: 0.5060
Epoch 33/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.9488 - accuracy: 0.4805 - val_loss: 3.7647 

2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8177 - accuracy: 0.4805 - val_loss: 3.6586 - val_accuracy: 0.5060
Epoch 83/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8110 - accuracy: 0.4805 - val_loss: 3.6572 - val_accuracy: 0.5060
Epoch 84/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7969 - accuracy: 0.4805 - val_loss: 3.6555 - val_accuracy: 0.5060
Epoch 85/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8063 - accuracy: 0.4805 - val_loss: 3.6546 - val_accuracy: 0.5060
Epoch 86/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8120 - accuracy: 0.4805 - val_loss: 3.6536 - val_accuracy: 0.5060
Epoch 87/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8118 - accuracy: 0.4805 - val_loss: 3.6523 - val_accuracy: 0.5060
Epoch 88/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8045 - accuracy: 0.4805 - val_loss: 3.6508 

2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8423 - accuracy: 0.4840 - val_loss: 3.7860 - val_accuracy: 0.4920
Epoch 37/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8394 - accuracy: 0.4840 - val_loss: 3.7823 - val_accuracy: 0.4920
Epoch 38/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8384 - accuracy: 0.4840 - val_loss: 3.7790 - val_accuracy: 0.4920
Epoch 39/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8269 - accuracy: 0.4840 - val_loss: 3.7758 - val_accuracy: 0.4920
Epoch 40/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8283 - accuracy: 0.4840 - val_loss: 3.7729 - val_accuracy: 0.4920
Epoch 41/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8325 - accuracy: 0.4840 - val_loss: 3.7703 - val_accuracy: 0.4920
Epoch 42/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8340 - accuracy: 0.4840 - val_loss: 3.7676 

2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7928 - accuracy: 0.4840 - val_loss: 3.6950 - val_accuracy: 0.4920
Epoch 92/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8067 - accuracy: 0.4840 - val_loss: 3.6939 - val_accuracy: 0.4920
Epoch 93/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7880 - accuracy: 0.4840 - val_loss: 3.6922 - val_accuracy: 0.4920
Epoch 94/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8050 - accuracy: 0.4840 - val_loss: 3.6912 - val_accuracy: 0.4920
Epoch 95/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.8048 - accuracy: 0.4840 - val_loss: 3.6899 - val_accuracy: 0.4920
Epoch 96/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7980 - accuracy: 0.4840 - val_loss: 3.6883 - val_accuracy: 0.4920
Epoch 97/100
2000/2000 [==============================] - 2s 1ms/sample - loss: 2.7994 - accuracy: 0.4840 - val_loss: 3.6870 

In [18]:
# # 모델 5번: transformer
# # https://www.tensorflow.org/tutorials/text/transformer

# class Transformer(tf.keras.Model):
#     def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
#                target_vocab_size, pe_input, pe_target, rate=0.1):
#         super(Transformer, self).__init__()

#         self.tokenizer = Encoder(num_layers, d_model, num_heads, dff, 
#                                input_vocab_size, pe_input, rate)

#         self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
#                                target_vocab_size, pe_target, rate)

#         self.final_layer = tf.keras.layers.Dense(target_vocab_size)

#     def call(self, inp, tar, training, enc_padding_mask, 
#            look_ahead_mask, dec_padding_mask):

#         enc_output = self.tokenizer(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

#         # dec_output.shape == (batch_size, tar_seq_len, d_model)
#         dec_output, attention_weights = self.decoder(
#             tar, enc_output, training, look_ahead_mask, dec_padding_mask)

#         final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

#         return final_output, attention_weights


# def build_transformer(split_num, train, target, test, rnd):
#     # return train pred prob and test pred prob 
#     train_pred, test_pred = np.zeros((train.shape[0], 1)), np.zeros((test.shape[0], 1))

#     skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
#     for train_idx, val_idx in skf.split(train, target):

#         # split train, validation set
#         X = train[train_idx]
#         y = target[train_idx]
#         valid_x = train[val_idx]
#         valid_y = target[val_idx]

#         #가벼운 모델 생성
#         model = Sequential()
#         model.add(LSTM(32, input_shape=(600,6)))
#         model.add(Dense(128, activation='relu'))
#         model.add(Dense(61, activation='softmax'))

#         model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#         model.fit(X,y, epochs=30, batch_size=128, validation_split=0.2)
        
#         # save feat
#         submission.iloc[:,1:]=model.predict(test_X)
#         train_pred[val_idx] = model.predict(valid_x).reshape(-1,1)
#         test_pred += model.predict(test).reshape(-1,1)/split_num
        
#         # release
#         del model
#         gc.collect()
#         print('------------------')
        
#     return train_pred, test_pred

# transformer_train1, transformer_test1 = build_transformer(5, train, train_y, test, 1)
# transformer_train2, transformer_test2 = build_transformer(5, train, train_y, test, 2)

In [19]:
xgb1_onehot = np.argmax(xgb_train1, axis=1).reshape(-1,1)
xgb2_onehot = np.argmax(xgb_train2, axis=1).reshape(-1,1)
catboost1_onehot = np.argmax(catboost_train1, axis=1).reshape(-1,1)
catboost2_onehot = np.argmax(catboost_train2, axis=1).reshape(-1,1)
cnn1_onehot = np.argmax(cnn_train1, axis=1).reshape(-1,1)
cnn2_onehot = np.argmax(cnn_train2, axis=1).reshape(-1,1)

xgb1_onehot_test = np.argmax(xgb_test1, axis=1).reshape(-1,1)
xgb2_onehot_test = np.argmax(xgb_test2, axis=1).reshape(-1,1)
catboost1_onehot_test = np.argmax(catboost_test1, axis=1).reshape(-1,1)
catboost2_onehot_test = np.argmax(catboost_test2, axis=1).reshape(-1,1)
cnn1_onehot_test = np.argmax(cnn_test1, axis=1).reshape(-1,1)
cnn2_onehot_test = np.argmax(cnn_test2, axis=1).reshape(-1,1)

train_final = np.hstack([xgb1_onehot, xgb2_onehot,
                         catboost1_onehot, catboost2_onehot,
#                          lstm_train1, lstm_train2,
                         cnn1_onehot, cnn2_onehot])

test_final = np.hstack([xgb1_onehot_test, xgb2_onehot_test,
                        catboost1_onehot_test, catboost2_onehot_test,
#                         lstm_test1, lstm_test2,
                        cnn1_onehot_test, cnn2_onehot_test])

print(train_final.shape)
print(test_final.shape)

# https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221343373342&proxyReferer=https:%2F%2Fwww.google.com%2F

(3125, 6)
(782, 6)


In [27]:
# 최종 앙상블

def ensemble_xgb(split_num, train, target, test):

    test_pred = np.zeros((test.shape[0], 61))
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 12,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                }
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=2021)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target.iloc[train_idx]
        valid_x = train[val_idx]
        valid_y = target.iloc[val_idx]
        
        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
    
    sample_submssion = pd.read_csv(path + 'sample_submission.csv')
    sample_submssion.iloc[:,1:] = test_pred
    sample_submssion.to_csv("ensemble.csv", index = False)
    
    print(sample_submssion)
    
ensemble_xgb(5, train_final, ml_label, test_final)

[0]	train-mlogloss:3.59783	valid-mlogloss:3.60179
[100]	train-mlogloss:1.23838	valid-mlogloss:1.48832
[173]	train-mlogloss:1.14565	valid-mlogloss:1.49086
------------------
[0]	train-mlogloss:3.60263	valid-mlogloss:3.61267
[100]	train-mlogloss:1.22994	valid-mlogloss:1.52272
[176]	train-mlogloss:1.13704	valid-mlogloss:1.52951
------------------
[0]	train-mlogloss:3.60738	valid-mlogloss:3.61075
[100]	train-mlogloss:1.25926	valid-mlogloss:1.40497
[195]	train-mlogloss:1.15417	valid-mlogloss:1.40005
------------------
[0]	train-mlogloss:3.60012	valid-mlogloss:3.60772
[100]	train-mlogloss:1.25102	valid-mlogloss:1.47219
[190]	train-mlogloss:1.15331	valid-mlogloss:1.46661
------------------
[0]	train-mlogloss:3.60298	valid-mlogloss:3.59853
[100]	train-mlogloss:1.26126	valid-mlogloss:1.40048
[180]	train-mlogloss:1.16569	valid-mlogloss:1.40216
------------------
       id         0         1         2         3         4         5  \
0    3125  0.007827  0.005372  0.001812  0.003146  0.001994  0